# Amazon SageMaker Clarifyを用いたモデルバイアスの検知


## Amazon Science: _[How Clarify helps machine learning developers detect unintended bias](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)_ 

[<img src="img/amazon_science_clarify.png"  width="100%" align="left">](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)

# 用語集

* **バイアス**:
年齢や収入層などの異なるグループ間で、学習データやモデルの予測動作に生じる不均衡のこと。バイアスは、モデルの学習に使用したデータやアルゴリズムから生じることがあります。例えば、MLモデルが主に中高年のデータで学習された場合、若年層や高齢者に対する予測を行う際に精度が低くなる可能性があります。

* **バイアスメトリクス**: 
潜在的なバイアスの度合いを示す数値を返す関数。

* **バイアスレポート**:
分析対象のデータセット、またはデータセットとモデルの組み合わせに対するバイアスメトリクスのコレクション。

* **ラベル**:
機械学習モデルのトレーニングのターゲットとなる特徴量。

* **ポジティブラベル値**:
サンプル内の特定の人口集団（年代、性別など）でよく観測されるラベル値。言い換えれば、サンプルがポジティブな結果を持つことを示しています。

* **ネガティブラベル値**:
サンプル内の特定の人口集団（年代、性別など）であまり観測されないラベル値。言い換えれば、サンプルがネガティブな結果を持つことを示しています。

* **ファセット**:
バイアスの分析対象となる属性を含むカラムまたは特徴量のこと。（訳注: 例えばデータセットに男女間の差異がないかを分析したい場合は、「性別」のカラムがファセット（側面）となります。）

* **ファセット値**:
バイアスが含まれ得る属性の特徴値。

# トレーニング後バイアスメトリクス
https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html

* **予測正例ラベル比率の差、Difference in Positive Proportions in Predicted Labels (DPPL)**:
有利なファセットaと不利なファセットdの間の正例の予測の比率の差を測定。

* **異種の影響度、Disparate Impact (DI)**:
有利なファセットaと不利なファセットdの予測されたラベル比率同士の割合を測定。

* **条件付き受け入れの差、Difference in Conditional Acceptance (DCAcc)**:
実際のポジティブラベルの数とモデルによって予測されたポジティブラベルの数の比率をファセット間で比較し、同じであるかどうかを評価。

* **条件付き拒否の差、Difference in Conditional Rejection (DCR)**:
実際のネガティブラベルの数とモデルによって予測されたネガティブラベルの数の比率をファセット間で比較し、同じであるかどうかを評価。

* **再現率の差、Recall Difference (RD)**:
有利なファセットと不利なファセットに対するモデルの再現率（recall）を比較。

* **受け入れ率の差、Difference in Acceptance Rates (DAR)**:
有利なファセットと不利なファセットとで、予測陽性（TP + FP）に対する真陽性（TP）の比率の差を測定。

* **拒否率の差、Difference in Rejection Rates (DRR)**:
有利なファセットと不利なファセットとで、真陰性（TN）と予測陰性（TN + FN）の比率の差を測定。

* **正確度の差、Accuracy Difference (AD)**:
有利なファセットと不利なファセットとで、正確度の差を測定。

* **処理の平等性、Treatment Equality (TE)**:
有利なファセットと不利なファセットの間の偽陽性と偽陰性の比率の差を測定。

* **予測ラベルにおける条件付き人口統計学的格差、Conditional Demographic Disparity in Predicted Labels (CDDPL)**:
ファセット全体だけでなく、サブグループごとの予測ラベルの格差を測定。

* **反実例のフリップテスト、Counterfactual Fliptest (FT)**:
ファセットdの各サンプルと類似したファセットaのサンプルを調べ、異なるモデル予測を持っているかどうかを評価します。


In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

# バイアス分析用のテストデータ

テストデータは、モデル入力と合うようにJSONLinesフォーマットで作成しました。

In [ ]:
test_data_bias_path = "./data-clarify/test_data_bias.jsonl"

In [ ]:
!head -n 1 $test_data_bias_path

### データのアップロード

In [ ]:
test_data_bias_s3_uri = sess.upload_data(bucket=bucket, key_prefix="bias/test_data_bias", path=test_data_bias_path)
test_data_bias_s3_uri

In [ ]:
!aws s3 ls $test_data_bias_s3_uri

In [ ]:
%store test_data_bias_s3_uri

# トレーニング後モデルバイアス分析を実行

In [ ]:
%store -r pipeline_name

In [ ]:
print(pipeline_name)

In [ ]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

# パイプライン実行ステップを表示


In [ ]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

In [ ]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

# 作成したモデルを確認
_注意:  トレーニングしたモデルが Evaluation ステップをパス（> 正確度の閾値）していない場合、作成されません。_

In [ ]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

pipeline_model_name = model_arn.split("/")[-1]
print(pipeline_model_name)

# SageMakerClarifyProcessor

In [ ]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.c5.2xlarge", 
    sagemaker_session=sess
)

# DataConfig と ModelConfig を書く

`DataConfig` オブジェクトはデータの入出力に関するいくつかの基本的な情報をClarifyに伝えるものです。
入力データセットをどこに置くか、出力をどこに保存するか、対象となるカラム（`label`）、ヘッダー名、データセットの種類などを指定します。

同様に、`ModelConfig` オブジェクトはトレーニングしたモデルに関する情報を設定し、`ModelPredictedLabelConfig` は予測値のフォーマットに関する情報をClarifyに伝えます。 

**注意**: SageMaker Clarifyでは、本番稼働中のモデルへの追加トラフィックを避けるために、処理実行時に専用のエンドポイントを立ち上げたり、停止させたりします。
`ModelConfig` は、Clarifyの処理中にモデルを実行するために使用する、希望のインスタンスタイプとインスタンス数を指定します。

## DataConfig

In [ ]:
bias_report_prefix = "bias/report-{}".format(pipeline_model_name)

bias_report_output_path = "s3://{}/{}".format(bucket, bias_report_prefix)

data_config = clarify.DataConfig(
    s3_data_input_path=test_data_bias_s3_uri,
    s3_output_path=bias_report_output_path,
    label="star_rating",
    features="features",
    # label must be last, features in exact order as passed into model
    headers=["review_body", "product_category", "star_rating"],
    dataset_type="application/jsonlines",
)

## ModelConfig

In [ ]:
model_config = clarify.ModelConfig(
    model_name=pipeline_model_name,
    instance_type="ml.m5.4xlarge",
    instance_count=1,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    # {"features": ["the worst", "Digital_Software"]}
    content_template='{"features":$features}',
)

## _注意: `label` にはモデル予測結果のJSONのキーをセットしています。_

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(label="predicted_label")

## BiasConfig

In [ ]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[
        5,
        4,
    ],  # カテゴリカルなデータ型の場合は「ポジティブ」なラベルを指定し、連続値の場合は閾値を指定する。
    facet_name="product_category",
)

# Clarifyジョブを実行

In [ ]:
clarify_processor.run_post_training_bias(
    data_config=data_config,
    data_bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    #    methods='all', # FlipTest requires all columns to be numeric
    methods=["DPPL", "DI", "DCA", "DCR", "RD", "DAR", "DRR", "AD", "TE"],
    wait=False,
    logs=False,
)

In [ ]:
run_post_training_bias_processing_job_name = clarify_processor.latest_job.job_name
run_post_training_bias_processing_job_name

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_post_training_bias_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?prefix={}/">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, bias_report_prefix
        )
    )
)

In [ ]:
from pprint import pprint

running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=run_post_training_bias_processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

pprint(processing_job_description)

In [ ]:
running_processor.wait(logs=False)

# S3からレポートをダウンロード

In [ ]:
!aws s3 ls $bias_report_output_path/

In [ ]:
!aws s3 cp --recursive $bias_report_output_path ./generated_bias_report/

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./generated_bias_report/report.html">Bias Report</a></b>'))

# Studio内でバイアスレポートを閲覧
StudioではExperiments and trialsから結果を閲覧できます。

<img src="img/bias_report.gif">

それぞれのバイアスメトリクスに対する詳細な説明とともにサンプルが記載されています。

<img src="img/bias_detail.gif">

また、便利なテーブルとして結果のサマリーを取得できます。

<img src="img/bias_report_chart.gif">

# リソースを解放

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>